In [175]:
import pickle
import json
import numpy as np
from operator import itemgetter
from nltk.tokenize import word_tokenize
import re
import string

In [176]:
# drones
tfhub_data_path = '/home/olga/Documents/drones_paragraph_ranking.pkl'
rnet_data_path = '/home/olga/Documents/r_net_ranker_scores.pckl'
# tfidf_data_path = '/media/olga/Data/projects/ODQA/data/PMEF/ranker1_drones_all_paragraphs.json'
tfidf_data_path = '/media/olga/Data/projects/iPavlov/DeepPavlov/download/pmef/ensemble_custom_drones.json'

#google
# tfhub_data_path = '/home/olga/Documents/wiki_chunks_tfhub.pkl'
# rnet_data_path = '/home/olga/Documents/wiki_r_net_scores.pckl'
# tfidf_data_path = '/media/olga/Data/projects/iPavlov/DeepPavlov/download/pmef/ensemble_google.json'
# tfidf_data_path = '/media/olga/Data/projects/iPavlov/DeepPavlov/download/pmef/ensemble_custom_google.json'


#git
# tfhub_data_path = '/home/olga/Documents/git_paragraph_ranking.pkl'
# rnet_data_path = '/home/olga/Documents/git_r_net_scores.pckl'
# tfidf_data_path = '/media/olga/Data/projects/iPavlov/DeepPavlov/download/pmef/ensemble_custom_git.json'

# drones #2
# tfhub_data_path = '/home/olga/Documents/drone_paragraph_ranking.pkl'
# rnet_data_path = '/home/olga/Documents/drones_r_net_scores.pckl'
# tfidf_data_path = '/media/olga/Data/projects/iPavlov/DeepPavlov/download/pmef/ensemble_drones.json'

# hottabych
# tfhub_data_path = '/home/olga/Documents/hottabych_paragraph_ranking.pkl'
# rnet_data_path = '/home/olga/Documents/hottabych_r_net_scores.pckl'
# tfidf_data_path = '/media/olga/Data/projects/iPavlov/DeepPavlov/download/pmef/ensemble_hottabych.json'

In [177]:
def load_pickle(path):
    with open(path, 'rb') as fin:
        return pickle.load(fin)

In [178]:
def load_json(path):
    with open(path) as fin:
        return json.load(fin)

In [179]:
tfhub_data = load_pickle(tfhub_data_path)
rnet_data = load_pickle(rnet_data_path)
tfidf_data = load_json(tfidf_data_path)

In [180]:
len(tfhub_data)

149

In [181]:
len(rnet_data)

137

In [182]:
len(tfidf_data)

149

In [183]:
tfidf_data[0:2]

[['what country manufactured the helicopter?',
  'Russia\n',
  [['Sberbank accounts for 28.9% of the total banking assets of the country (as of January 1, 2018), and its active customers are 60% of the population. Sberbank is among the top 5 largest employers of Russia. The Bank is the main creditor of the Russian economy and holds the largest share in the deposit market. It accounts for 46.1% of household deposits, 40.5% of loans to individuals and 32.4% of loans to legal entities (as of January 1, 2018). Sberbank consists of 14 territorial banks and 14.2 thousand branches in the Russian Federation. The Bank\'s foreign network consists of subsidiaries, branches and representative offices in the CIS, Central and Eastern Europe, Turkey, the United Kingdom, the United States and other countries. The number of active retail customers of Sberbank in Russia exceeds 86 million people, the number of corporate clients of the Group is more than 2 million in Russia. The range of Sberbank service

In [184]:
rnet_data[0]

('what country manufactured the helicopter?',
 'Russia',
 [(0.9978230595588684,
   'Helicopters of Russia JSC (part of Rostekh State Corporation) is one of the world leaders in the helicopter industry, the only developer and manufacturer of helicopters in Russia. The holding was established in 2007. The head office is located in Moscow. The holding includes five helicopter factories, two design bureaus, as well as enterprises for the production and maintenance of components, aircraft repair plants and a service company that provides after-sales support in Russia and abroad.'),
  (0.8298145532608032,
   'The St. Petersburg International Economic Forum is scheduled to sign a Memorandum of Cooperation between PJSC Sberbank and JSC Helicopters of Russia, within the framework of this memorandum, it is planned to establish partnerships aimed at creating a federal logistics operator using unmanned aerial systems of a helicopter type.'),
  (0.792402982711792,
   "The Bank's foreign network con

In [185]:
all_questions = [item[0] for item in rnet_data]
len(all_questions)

137

In [186]:
tfhub_data = [item for item in tfhub_data if item[0] in all_questions]
len(tfhub_data)

149

In [187]:
tfidf_data = [item for item in tfidf_data if item[0] in all_questions]
len(tfidf_data)

149

In [188]:
# make paragraph: id map

all_pars = []
for instance in rnet_data:
    pars = instance[2]
    all_pars += [p[1] for p in pars]

all_pars = list(set(all_pars))

In [189]:
len(all_pars)

58

In [190]:
par2id = {all_pars[i]: i for i in range(len(all_pars))}

In [191]:
id2par = {v: k for k, v in par2id.items()}

In [192]:
all_pars_tfhub = []
for instance in tfhub_data:
    pars = instance[2]
    all_pars_tfhub += [p[0] for p in pars]

all_pars_tfhub = list(set(all_pars_tfhub))

for p in all_pars_tfhub:
    if p not in par2id.keys():
        print(p)
else:
    print('All pars exist.')

All pars exist.


In [195]:
all_pars_tfidf = []
for instance in tfidf_data:
    pars = instance[2]
    print(pars)
    all_pars_tfidf += [unicodedata.normalize("NFD", p[0]) for p in pars]

all_pars_tfidf = list(set(all_pars_tfidf))
print(all_pars_tfidf)

# for p in all_pars_tfidf:
#     if p not in par2id.keys():
#         print(p)
#         print('\n')
# else:
#     print('All pars exist.')

[['Sberbank accounts for 28.9% of the total banking assets of the country (as of January 1, 2018), and its active customers are 60% of the population. Sberbank is among the top 5 largest employers of Russia. The Bank is the main creditor of the Russian economy and holds the largest share in the deposit market. It accounts for 46.1% of household deposits, 40.5% of loans to individuals and 32.4% of loans to legal entities (as of January 1, 2018). Sberbank consists of 14 territorial banks and 14.2 thousand branches in the Russian Federation. The Bank\'s foreign network consists of subsidiaries, branches and representative offices in the CIS, Central and Eastern Europe, Turkey, the United Kingdom, the United States and other countries. The number of active retail customers of Sberbank in Russia exceeds 86 million people, the number of corporate clients of the Group is more than 2 million in Russia. The range of Sberbank services for retail customers is maximally wide: from traditional depo

In [110]:
all_questions

['what country manufactured the helicopter?',
 'in which year was the helicopter created?',
 'weight of a drones?',
 'how long does the helicopter work?',
 'What are the dimensions of this model of unmanned helicopter?',
 'where you can buy?',
 'where you can buy a helicopter?',
 'where is the helicopter?',
 'where is it?',
 'What is the carrying capacity of a helicopter?',
 'What is the weight of the helicopter?',
 'What is the mass of the helicopter?',
 'What time can be in the air?',
 'what amount of payload is it capable of carrying?',
 'what is the weight of the helicopter?',
 'how long can the helicopter be in the air?',
 'how long is the helicopter capable of being in the air?',
 'how long is the duration of a helicopter flight?',
 'how much does the helicopter weigh?',
 'how long can a it stay in the air without moving around ?',
 'how many types of unmanned helicopters are there?',
 'how many types does it have?',
 'how many wings it has?',
 'what is the average weight of an u

In [111]:
# modify anaphopric questions

# anaphora_tokens = ['it', 'this', 'these', 'they']
# new_questions = []
# prev_tokens = []
# for i in range(len(all_questions)):
#     q = all_questions[i]
#     cur_tokens = word_tokenize(q)
#     if (any(t in cur_tokens for t in anaphora_tokens)):
#         if prev_tokens:
#             new_question = ' '.join(set(cur_tokens + prev_tokens))
#             print(new_question)
#             new_questions.append(new_question)
#         else:
#             new_questions.append(all_questions[i])
#     else:
#         new_questions.append(all_questions[i])
#         prev_tokens = cur_tokens
    
    

In [112]:
len(all_questions)

137

In [83]:
# len(new_questions)

In [113]:
all_answers = [item[1] for item in rnet_data]

In [167]:
# pp = "Sberbank accounts for 28.9% of the total banking assets of the country (as of January 1, 2018), and its active customers are 60% of the population. Sberbank is among the top 5 largest employers of Russia. The Bank is the main creditor of the Russian economy and holds the largest share in the deposit market. It accounts for 46.1% of household deposits, 40.5% of loans to individuals and 32.4% of loans to legal entities (as of January 1, 2018). Sberbank consists of 14 territorial banks and 14.2 thousand branches in the Russian Federation. The Bank's foreign network consists of subsidiaries, branches and representative offices in the CIS, Central and Eastern Europe, Turkey, the United Kingdom, the United States and other countries. The number of active retail customers of Sberbank in Russia exceeds 86 million people, the number of corporate clients of the Group is more than 2 million in Russia. The range of Sberbank services for retail customers is maximally wide: from traditional deposits and various types of lending to bank cards, money transfers, bank insurance and brokerage services. The Bank serves all corporate customer groups, paying special attention to remote service channels. Online bank for legal entities, \"Sberbank Business Online\", monthly use more than 1.5 million customers - legal entities. In recent years, Sberbank has been at the forefront of technological and innovative changes in the country. We successfully transform our own technological platform and create the largest industrial IT system based on the most modern technologies. In addition, we are actively developing innovative technologies, such as artificial intelligence, machine learning, blockage, robotics and others. On the basis of Sberbank, we do not simply introduce new technologies into our business processes, but also invest in their development and popularization on a national scale. In 2017, Sberbank adopted the Development Strategy until 2020, the main goal of which is to implement initiatives that will allow the Bank to reach a new level of competitiveness, enabling it to compete with global technology companies while remaining the best bank for the population and business. Helicopters of Russia JSC (part of Rostekh State Corporation) is one of the world leaders in the helicopter industry, the only developer and manufacturer of helicopters in Russia. The holding was established in 2007. The head office is located in Moscow. The holding includes five helicopter factories, two design bureaus, as well as enterprises for the production and maintenance of components, aircraft repair plants and a service company that provides after-sales support in Russia and abroad. The buyers of the holding's products are the Ministry of Defense of Russia, the Ministry of Internal Affairs of Russia, EMERCOM of Russia, other government customers, airlines Gazprom and UTair, large Russian and foreign companies. In 2016, the revenue of Russian Helicopters under IFRS amounted to 214.3 billion rubles, the volume of deliveries amounted to 189 helicopters\n\nState Corporation Rostekh is a Russian corporation established in 2007 to promote the development, production and export of high-tech industrial products for civilian and military use."

import unicodedata
for i in range(len(tfidf_data)):
    for par, score in tfidf_data[i][2]:
        tfidf_data[i][2] = [unicodedata.normalize('NFD', str(par)), score]

In [168]:
tfidf_data[0:2]

[['what country manufactured the helicopter?',
  'Russia\n',
  ["The average annual growth rate of the market (CAGR) during the forecast period is expected at 34%. The authors of the study noted that the share of drones for industrial monitoring during this period will remain the largest - about 30%. 7% of drones will have to enter the agricultural market. For commercial delivery, due to the high costs of operations, only 1% of drones will be required. Representatives of the company Tractica predict that the revenue from the supply of drones and the provision of services will grow from $ 792 million in 2017 to $ 12.6 billion by 2025. The company's report states that most of the revenue comes from equipment sales. Then this share will shift towards services and operations. Analysts note the following key trends in the development of the market for unmanned aerial vehicles:\n\n• Drones are actively being introduced into segments that are now serviced by satellites and manned aircraft, wh

In [169]:

mean_rankers_data = []

for q, a in zip(all_questions, all_answers):
#     print(q)
    tfhub_par_scores = [par_scores for quest, ans, par_scores in tfhub_data if quest==q][0]
    tfhub_par_id_scores = {par2id[par]: score for par, score in tfhub_par_scores}
    rnet_par_scores = [par_scores for quest, ans, par_scores in rnet_data if quest==q][0]
    rnet_par_id_scores = {par2id[par]: score for score, par in rnet_par_scores}
#     try:
    tfidf_par_scores = [par_scores for quest, ans, par_scores in tfidf_data if quest==q]
#     except IndexError:
#         print(q, a)
#     print(tfhub_par_scores[0])

    tfidf_par_id_scores = {par2id[par]: score for par, score in tfidf_par_scores}
    
    norm = np.linalg.norm(list(tfidf_par_id_scores.values()))
    tfidf_par_id_scores = {k: v / norm for k, v in tfidf_par_id_scores.items()}
    
#     print(tfhub_par_id_scores)
#     print(rnet_par_id_scores)
#     print(tfidf_par_id_scores)
    
    idx2mean = []
    for idx in par2id.values():
        tfhub_score = tfhub_par_id_scores[idx]
        rnet_score = rnet_par_id_scores[idx]
        tfidf_score = tfidf_par_id_scores[idx]
        
        mean_score = np.mean([rnet_score, tfidf_score])
        idx2mean.append((idx, mean_score))
        
    idx2mean.sort(key=itemgetter(1))
    idx2mean = idx2mean[::-1]
    
    mean_rankers_data.append((q, a, idx2mean))
    

KeyError: "The average annual growth rate of the market (CAGR) during the forecast period is expected at 34%. The authors of the study noted that the share of drones for industrial monitoring during this period will remain the largest - about 30%. 7% of drones will have to enter the agricultural market. For commercial delivery, due to the high costs of operations, only 1% of drones will be required. Representatives of the company Tractica predict that the revenue from the supply of drones and the provision of services will grow from $ 792 million in 2017 to $ 12.6 billion by 2025. The company's report states that most of the revenue comes from equipment sales. Then this share will shift towards services and operations. Analysts note the following key trends in the development of the market for unmanned aerial vehicles:\n\n• Drones are actively being introduced into segments that are now serviced by satellites and manned aircraft, while the demand for high-quality aerial survey data is 

In [145]:
tfidf_par_scores[1]

0.6759020934108961

In [135]:
print([k for k, v in par2id.items() if k.startswith('Sberbank accounts')])

[]


In [86]:
id2par[2]

'*/||\'`\ndo\n\tif [ $branch = test -o $branch = release ]\n\tthen\n\t\tcontinue\n\tfi\n\n\techo -n $gb ======= $branch ====== $restore " "\n\tstatus=\n\tfor ref in test release origin/master\n\tdo\n\t\tif [ `git rev-list $ref..$branch | wc -c` -gt 0 ]\n\t\tthen\n\t\t\tstatus=$status${ref:0:1}\n\t\tfi\n\tdone\n\tcase $status in\n\ttrl)\n\t\techo $rb Need to pull into test $restore\n\t\t;;\n\trl)\n\t\techo "In test"\n\t\t;;\n\tl)\n\t\techo "Waiting for linus"\n\t\t;;\n\t"")\n\t\techo $rb All done $restore\n\t\t;;\n\t*)\n\t\techo $rb "<$status>" $restore\n\t\t;;\n\tesac\n\tgit log origin/master..$branch | git shortlog\ndone\n-------------------------------------------------\n\n\n[[cleaning-up-history]]\nRewriting history and maintaining patch series\n==============================================\n\nNormally commits are only added to a project, never taken away or\nreplaced. Git is designed with this assumption, and violating it will\ncause Git\'s merge machinery (for example) to do the 

In [87]:
mean_rankers_data[0]

('What is git?',
 'Git is a fast distributed revision control system.',
 [(27, 0.65767170442139022),
  (5, 0.6355310226777684),
  (8, 0.6313367928365361),
  (33, 0.59609051002060287),
  (16, 0.5543399597505223),
  (18, 0.49999219152015062),
  (15, 0.49450936886351915),
  (22, 0.46006695879494058),
  (29, 0.4381887163022648),
  (6, 0.42132815691512437),
  (34, 0.41887909146827074),
  (7, 0.39204001399558397),
  (23, 0.38015568229239793),
  (12, 0.37956702682059618),
  (2, 0.37634035917800279),
  (42, 0.37599015208762498),
  (14, 0.37351158234160753),
  (37, 0.35583606335204454),
  (13, 0.34705975624602647),
  (36, 0.33824220272582384),
  (11, 0.33188933106940599),
  (24, 0.32969959033523905),
  (28, 0.32826843830626817),
  (26, 0.32495200606864305),
  (25, 0.32441642853301378),
  (35, 0.31509539457839342),
  (31, 0.31157359453719469),
  (19, 0.26398837539237352),
  (17, 0.2636856732038817),
  (10, 0.24259433123153062),
  (43, 0.22779086205046983),
  (1, 0.22657749029677721),
  (40, 0.22

In [88]:
def instance_score1(answer, texts):
    for doc_text in texts:
        if doc_text.find(answer) != -1:
            return 1
    return 0

def instance_score(a, texts):
    regex = re.compile('[%s]' % re.escape(string.punctuation))

    a = regex.sub(' ', a)
    a = re.sub(" +", " ", a)
    a = a.replace('\n', ' ')
    for doc_text in texts:
        doc_text = regex.sub(' ', doc_text)
        doc_text = re.sub(" +", " ", doc_text)
        doc_text = doc_text.replace('\n', ' ')
        if doc_text.find(a.strip()) != -1:
            return 1
    return 0

In [89]:
recall_mapping = {}
for n in range(1, len(id2par) +1):
    correct_answers = 0
    for instance in mean_rankers_data:
        top_n_ids = [pair[0] for pair in instance[2]][:n]
        top_n_texts = [id2par[j] for j in top_n_ids]
        top_n_texts = [text.lower() for text in top_n_texts]
        formatted_answer = instance[1].strip().lower()
        correct_answers += instance_score(formatted_answer, top_n_texts)
    recall = correct_answers / len(rnet_data)
    recall_mapping[n] = recall

In [90]:
recall_mapping

{1: 0.5666666666666667,
 2: 0.6666666666666666,
 3: 0.7333333333333333,
 4: 0.7666666666666667,
 5: 0.7666666666666667,
 6: 0.8,
 7: 0.8,
 8: 0.8333333333333334,
 9: 0.8333333333333334,
 10: 0.8333333333333334,
 11: 0.8333333333333334,
 12: 0.8333333333333334,
 13: 0.8333333333333334,
 14: 0.8333333333333334,
 15: 0.8333333333333334,
 16: 0.8333333333333334,
 17: 0.8333333333333334,
 18: 0.8666666666666667,
 19: 0.8666666666666667,
 20: 0.8666666666666667,
 21: 0.8666666666666667,
 22: 0.9,
 23: 0.9,
 24: 0.9,
 25: 0.9,
 26: 0.9,
 27: 0.9,
 28: 0.9,
 29: 0.9,
 30: 0.9,
 31: 0.9,
 32: 0.9,
 33: 0.9,
 34: 0.9,
 35: 0.9,
 36: 0.9,
 37: 0.9,
 38: 0.9,
 39: 0.9,
 40: 0.9,
 41: 0.9,
 42: 0.9,
 43: 0.9,
 44: 0.9,
 45: 0.9,
 46: 0.9}

In [209]:
len(rnet_data)

40